### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [9]:
!pip install ollama


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model

In [11]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [12]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### Second concept: The Message

In [13]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(id='9e22e9a3-8c46-483c-b962-4cfaa2dcb618', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 12, 16, 15, 38, 53, 467057, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

### Third concept: The Agent

In [14]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

### Put it all together with on_messages

In [15]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

"Sure! London is lovely—just don’t forget your umbrella and your sense of humor! They'll come in handy for both the rain and the British sarcasm!"

### Let's make a local database of ticket prices

In [16]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [17]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [18]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [19]:
get_city_price("Rome")

499.0

In [20]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [21]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_t7jhrVR0oAYgn9XVUY7oawui', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_t7jhrVR0oAYgn9XVUY7oawui', is_error=False)]


'Ready for some tea time in London? A roundtrip ticket will only cost you $299! Just keep an eye out for those pesky pigeons! 🐦✈️'